In [127]:
import httplib
from nltk.stem.snowball import FrenchStemmer
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import urllib2
import random
import re
import requests
from httplib import IncompleteRead
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import sys  
#reload(sys)  
#sys.setdefaultencoding('latin-1')

In [128]:
departement=['Bakel','Bambey','Bignona','Birkelane','Bounkiling','Dagana','Dakar','Diourbel','Fatick','Foundiougne',
            'Gossas','Goudiry','Goudomp','Guediawaye','Guinguineo','Kaffrine','Kanel','Kaolack',
             'Kebemer','Kedougou','Kolda','Koumpentoum','Koungheul','Linguere','Louga','Mbour',
             'Malem Hodar','Matam','Mbacke','Medina Yoro Foulah','Nioro du Rip','Oussouye',
             'Pikine','Podor','Ranerou-Ferlo','Rufisque','Saint-Louis','st louis','Salemata','Saraya',
             'Sedhiou','Tambacounda','Thies','Tivaouane','Velingara','Ziguinchor']

arrondissement = ['Almadies','Agnam Civol','Baba Garage','Bandafassi','Brassou','Bala','Bele','Boghal','Bona',
                  'Boyngel Bamba','Bamba Thialene','Barkedji','Bembou','Bonkoto','Cas-Cas','Colobane','Coki',
                  'Darou Minam','Dar Salam','Dakateli','Dagoudane','Dodji','Dakar Plateau',
                  'Diakhao','Djilor','Djibanar','Djoulacolon','Diaroume','Darou Mousty','Dianke Makha','Diende',
                  'Djibabouya','Djiredji','Fafacourou','Fissel','Fongolembi', 'Fimela',
                  'Grand Dakar','Gamadji Sare','Guediawaye','Gniby',
                  'Kataba','Kael','Kabrousse','Katakel','Karatanba','Koussanar','Keur Moussa',
                  'Keniaba','Koulor','Koumbal','Kouthiaba Wolof','Keur Momar Sarr','Keur Mboucki',
                  'Lambaye','Lour Escale','Loudia Ouoloff','Ida Mouride',
                  'Mampatim','Makacolibantang','Mouderi','Mbane','Mbadakhoune','Missira Wadine','Missirah',
                  'Meouane','Mbediene','Mabo','Medina Sabakh',
                  'Ndindy','Ndoulo','Ndiedieng','Ndande','Ndame','Niodor','Ngoye','Ndorna','Niakhene',
                  'Niaming','Niayes','Ndiaye','Niakhar','Niaguis','Notto','Nyassia',
                  'Ogo','Orkadiere','Ouadiour', 'Parcelles Assainies','Pambal','Pakour','Paoskoto','Rao','Rufisque',
                  'Sagatta','Sagatta Dioloff','Sare Bidji','Sare Coly Salle','Sindian','Sangalkam','Sagna','Sakal',
                  'Sabodala','Sibassor','Sindia','Sessene','Salde','Thienaba','Thies',
                  'Taif','Tendouck','Tenghory','Tattaguine','Thiaroye','Toubacouta','Thille Boubacar',
                  'Velingara','Wouro Sidy','Wack Ngouna','Yang Yang']
month=['janvier','fevrier','mars','avril','mai','juin','juillet','aout','septembre','octobre','novembre','decembre']

### Load data

In [129]:
df= pd.read_csv("flood_event_story.csv",sep=";")
df.shape

(490, 4)

### Remove link that had not story

In [130]:
df['story']=df['story'].apply(lambda x: str(x).strip() )
df=df[ df['story']!='nan' ]
df.index= range(len(df.link))
df.shape

(332, 4)

### Remove newline from paragraph of each story

In [131]:
df['story']= df['story'].apply( lambda x: x.replace('\n',' ') )
df['story']= df['story'].apply( lambda x: x.replace('\t',' ') )
df['story']= df['story'].apply( lambda x: x.replace('.>>','') )
df['story']= df['story'].apply( lambda x: x.replace('"','') )
df['story']= df['story'].apply( lambda x: x.replace('«','') )
df['story']= df['story'].apply( lambda x: x.strip() )
df['story'].tail()

327    Sign in with a different account         Creat...
328    Sign in with a different account         Creat...
329    Sign in with a different account         Creat...
330    Sign in with a different account         Creat...
331    Sign in with a different account         Creat...
Name: story, dtype: object

### Give score of relevance and remove 0 score

In [132]:
#main subject on the link +1
df['relevancy'] = [1 if re.search("(inondation|inondes)",x) is not None else 0 for x in df.link  ]
# main subject on the content story +1
df['relevancy'] += [1 if re.search("(inondation|inondes|sinistre)",x) is not None else 0 for x in df.story]
#exact match on explicit group of words +2
df['relevancy'] += [2 if re.search("\\bsous les eaux\\b",x) is not None else 0 for x in df.story]
df['relevancy'] += [2 if re.search("\\binondés par\\b",x) is not None else 0 for x in df.story]
df['relevancy'] += [2 if re.search("\\binondations à\\b",x) is not None else 0 for x in df.story]
#other keywords that may be indicative
df['relevancy'] += [1 if re.search("\\commune de\\b",x) is not None else 0 for x in df.story]
df['relevancy'] += [1 if re.search("\\département de\\b",x) is not None else 0 for x in df.story]
df['relevancy'] += [1 if re.search("\\assainissement\\b",x) is not None else 0 for x in df.story]
df['relevancy'] += [1 if re.search("\\les eaux\\b",x) is not None else 0 for x in df.story]
df['relevancy'] += [0.5 if re.search("\\problème\\b",x) is not None else 0 for x in df.story]
df['relevancy'] += [0.5 if re.search("\\interpelle\\b",x) is not None else 0 for x in df.story]
# mention of any department on content +1:
df['relevancy'] +=[ 1 if x in departement else 0 for x in df.story ]
# mention of any arrondissement on content +1:
df['relevancy'] +=[ 1 if x in arrondissement else 0 for x in df.story  ]
df=df[df.relevancy>0]
df.index= range(len(df.link))
df.shape

(105, 5)

### Sort result for visualization

In [133]:
df[['story','link','relevancy']].sort_values(by=['relevancy'], ascending=False).head()

,story,link,relevancy
90,12/07/2015 à 19:38 22/02/2016 à 22:42 → Accue...,http://www.senenews.com/2015/07/12/kaffrine-so...,5.5
73,Quotidien d'informations du Senegal Voir tous ...,http://www.senegaltribune.com/articles/3263/1/...,5.0
57,L’eau a envahi la banlieue dakaroise. Les plui...,http://www.senegaltribune.com/articles/2889/1/...,4.5
5,Après les fortes pluies de la nuit du dimanche...,http://laviesenegalaise.com/inondations-a-mata...,4.5
6,Après les fortes pluies de la nuit du dimanche...,http://laviesenegalaise.com/inondations-a-mata...,4.5


### Get place (department or arrondissement )

In [134]:
df['place']=''
for i, j in enumerate(df['link']):
    x= [ k for k in departement if k.lower() in j  ] +[ k for k in arrondissement if k.lower() in j  ]
    if x!=[]:
        if len(x)==1:
            df.loc[i,'place']=str(x)
        if len(x)>1:
            df.loc[i,'place']="-".join(x)
    else:
        df.loc[i,'place']=''

### Search for place in "Story" content of each page

In [135]:
for i, j in enumerate(df['story']):
    x= [ k for k in departement if k.lower() in j and df.loc[i,'place']==''  ]
    x += [ k for k in arrondissement if k.lower() in j and df.loc[i,'place']==''  ]
    if x!=[]:
        if len(x)==1:
            df.loc[i,'place']=str(x)
        if len(x)>1:
            df.loc[i,'place']="-".join(x)

### Check duplicate records

In [136]:
# mean number of character of all the links
# slice based on median
#drop duplicates based on those shorten links
# but before decrease until duplicates appear
z=[len(x) for x in df.link]
median_link_index= int(round(np.median(z)))
L=len(df.link)
while L == len(df.link):
    df['link_median']= df['link'].apply(lambda x: x[:median_link_index])
    L=len(df.link_median.unique())
    median_link_index= median_link_index - 5
df=df.drop_duplicates(subset=["link_median"], inplace = False)

### Get keywords from links

* from links
* from story content

In [137]:
def handle_links(x):
    x= x.replace('//','-')
    x=x.replace('--','-')
    x=x.split('-')
    #ignore www & domaine name
    x= ' '.join(x[1:])
    x=x.replace(".html","")
    x=x.strip()
    return x
df['link_kwds']=df['link'].apply(lambda x: handle_links(x) )
df.link_kwds.head()

0    africaleadnews.com/changement climatique perte...
1    laviesenegalaise.com/effondrement de deux bati...
2    laviesenegalaise.com/effondrement de deux bati...
3    laviesenegalaise.com/fatick receptionne deux d...
4    laviesenegalaise.com/fatick receptionne deux d...
Name: link_kwds, dtype: object

### Remove french stop words from keywords links, story variables

In [138]:
df['link_kwds']= df['link_kwds'].apply(lambda x: ' '.join([ j for j in unicode(x,'utf-8').split(' ') if j not in stopwords.words('french')]))

In [139]:
df['story']= df['story'].apply(lambda x: ' '.join([ j for j in x.split(' ') if j not in stopwords.words('french')]))

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if __name__ == '__main__':


### Apply stemming to keywords links , story variables

In [140]:
df['link_kwds']= df['link_kwds'].apply(lambda x:FrenchStemmer().stem(x))
df['story']= df['story'].apply(lambda x:FrenchStemmer().stem(unicode(x,'utf-8')))
df['story']= df['story'].apply( lambda x: x.strip() )
df['story'].tail()

100    de fortes pluies à hauteur 29 millimètres tomb...
101    de fortes pluies à hauteur 29 millimètres tomb...
102    sep 22, 2010 oct 20, 2016 oct 20, 2016 oct 20,...
103    16/06/2016 xibaaru actualités, exclusivités 0 ...
104    16/06/2016 xibaaru actualités, exclusivités 0 ...
Name: story, dtype: object

### Loop again to get place name for those missing

In [141]:
for i, j in enumerate(df['link']):
    x= [ k for k in departement if k.lower() in j  ] +[ k for k in arrondissement if k.lower() in j  ]
    if x!=[] and df.loc[i,'place']=='':
        if len(x)==1:
            df.loc[i,'place']=str(x)
        if len(x)>1:
            df.loc[i,'place']="-".join(x)
######################################################################################################
for i, j in enumerate(df['link_kwds']):
    x= [ k for k in departement if k.lower() in j  ] +[ k for k in arrondissement if k.lower() in j ]
    if x!=[] and df.loc[i,'place']=='':
        if len(x)==1:
            df.loc[i,'place']=str(x)
        if len(x)>1:
            df.loc[i,'place']="-".join(x)

### drop those that do not have place

In [142]:
df=df[df.place!='']
df.index=range(len(df))
df.shape

(53, 8)

## Get year

In [143]:
df['date']=df['link'].apply( lambda x: [ z for z in range(2000,2016) if str(z) in x  ] )
df['date']=df['link_kwds'].apply( lambda x: [ z for z in range(2000,2016) if str(z) in x and x!=[] ] )
df['date']=df['story'].apply( lambda x: [ z for z in range(2000,2016) if str(z) in x and x!=[] ] )

In [144]:
df=df[df.relevancy>=1]
df.index= range(len(df))
df[['link','place','date']]

,link,place,date
0,http://africaleadnews.com/changement-climatiqu...,['Dakar'],[]
1,http://laviesenegalaise.com/effondrement-de-de...,['Tambacounda'],[]
2,http://laviesenegalaise.com/effondrement-de-de...,['Tambacounda'],[]
3,http://laviesenegalaise.com/fatick-receptionne...,['Fatick'],[2014]
4,http://laviesenegalaise.com/fatick-receptionne...,['Fatick'],[2014]
5,http://laviesenegalaise.com/inondations-a-mata...,['Matam'],[]
6,http://laviesenegalaise.com/inondations-a-mata...,['Matam'],[]
7,http://laviesenegalaise.com/tambacounda-dernie...,['Tambacounda'],[]
8,http://laviesenegalaise.com/tambacounda-dernie...,['Tambacounda'],[]
9,http://laviesenegalaise.com/urgent-kaffrine-so...,['Kaffrine'],[]
